In [1]:
import Environment
import parameters
pa = parameters.Parameters()
pa.num_nw = 5
pa.simulate_len = 50
pa.num_ex = 10
pa.new_job_rate = 1
pa.compute_dependent_parameters()
env = Environment.Allocation_Environment(pa, render=False, repre='image')
env.step(5)
env.step(5)
env.step(5)
env.step(5)
env.step(5)


Load On # 0 Resource Dimension Is  1.1594666666666666
Load On # 1 Resource Dimension Is  1.2198666666666667


(array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 -11.000000000000002,
 False,
 <Job.JobRecord at 0x1a692120610>)

In [3]:
job = env.job_backlog.backlog[0]


In [5]:
env.job_backlog.show()

+--------+------------+------------+------------+-------------+
| Job Id | Res Vector | Enter Time | Start Time | Finish Time |
+--------+------------+------------+------------+-------------+
|   1    |   [2 8]    |     2      |     -1     |      -1     |
|   2    |   [8 2]    |     3      |     -1     |      -1     |
|   2    |   [8 2]    |     3      |     -1     |      -1     |
|   3    |   [7 1]    |     4      |     -1     |      -1     |
|   3    |   [7 1]    |     4      |     -1     |      -1     |
|   3    |   [7 1]    |     4      |     -1     |      -1     |
|   4    |  [ 2 10]   |     5      |     -1     |      -1     |
|   4    |  [ 2 10]   |     5      |     -1     |      -1     |
|   4    |  [ 2 10]   |     5      |     -1     |      -1     |
|   4    |  [ 2 10]   |     5      |     -1     |      -1     |
+--------+------------+------------+------------+-------------+


In [ ]:
import numpy as np


In [ ]:
n

In [ ]:
import tensorflow
import gym
import pandas

In [ ]:
tensorflow.config.list_physical_devices()

In [ ]:
csv = pandas.read_csv("Sever.csv")

In [ ]:
ze =numpy.zeros([3,15])

In [ ]:
ze

In [ ]:
z1 = numpy.array([1,2,3])

In [ ]:
for i in range(1,100):
    on1 = numpy.ones([10])
    cpu = on1 * 10
    mem = on1 * 20
    numpy.concatenate([cpu,mem],axis=0).reshape([2,10])

In [ ]:
numpy.array([10,20]).repeat(10).reshape([2,10])

In [ ]:
request = [0,0]

In [ ]:
request.__len__()

In [ ]:
numpy.repeat(z1,15).reshape([z1.size,15])

In [ ]:
    class Server_State:
        # 剩余资源
        def __init__(self,server_def=[0,0],server_cost = [0,0], 
                            server_used = numpy.zeros([2,15]),time = 15) -> None:
            # 资源
            # 持有资源
            self.possess = numpy.array(server_def)
            # 已用资源
            self.used = server_used
            # 可用资源
            self.available = numpy.repeat(self.possess,time)\
                .reshape([self.possess.size,time]) - server_used
            self.cost = numpy.array(server_cost)
            pass
        def allocate(self,server_use = numpy.zeros([2,15])):
            self.available -= server_use
            print("{a}",self.available)

In [ ]:
st = Server_State([123,255],[1235,51555],numpy.ones([2,15]),15)

In [ ]:
st.allocate(numpy.ones([2,10])+numpy.zeros([2,15]))

In [ ]:
numpy.zeros([2,15])+numpy.ones([2,10])

In [ ]:
on1 = numpy.ones([2,10])

In [ ]:
numpy.concatenate((on1,numpy.zeros([2,5])),axis=1)

In [ ]:
st.available[0][0]

In [ ]:
a = [[0,1],[0]]
a[0].__len__()

In [ ]:
numpy.zeros([2,3])

In [ ]:
3*365

In [ ]:
73229/1095

In [ ]:
import numpy
a = numpy.zeros([4,5])
a.len

In [5]:
import job_distribution
dist = job_distribution.Dist(2,10,15)
nw_len,size = dist.bi_model_dist()
size


array([9., 2.])

In [1]:
import numpy as np
import Environment
env = Environment.Allocation_Environment()
env.nw_size_seq[0,0]

ValueError: could not broadcast input array from shape (2,) into shape (5,)

In [2]:
import sys
sys.path.append("..")

In [3]:
sys.path

['c:\\Users\\Yuming\\source\\repos\\ResMan',
 'C:\\Program Files\\Python311\\python311.zip',
 'C:\\Program Files\\Python311\\DLLs',
 'C:\\Program Files\\Python311\\Lib',
 'C:\\Program Files\\Python311',
 'c:\\Users\\Yuming\\source\\repos\\ResMan\\man',
 '',
 'c:\\Users\\Yuming\\source\\repos\\ResMan\\man\\Lib\\site-packages',
 'c:\\Users\\Yuming\\source\\repos\\ResMan\\man\\Lib\\site-packages\\win32',
 'c:\\Users\\Yuming\\source\\repos\\ResMan\\man\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\Yuming\\source\\repos\\ResMan\\man\\Lib\\site-packages\\Pythonwin',
 '..']

In [2]:
import parameters
import Environment
pa = parameters.Parameters()
env = Environment.Allocation_Environment(pa)


Load On # 0 Resource Dimension Is  0.6933333333333334
Load On # 1 Resource Dimension Is  1.7933333333333334


In [9]:
job = env.get_new_job_from_seq(0,0)
job.res_vec

array([1, 9])

In [3]:
import Environment
def test_image_representation():
    """
    Purpose: representation the job image
    """
    env = Environment.Allocation_Environment()
    return env.machine.canvas

In [5]:
canvas = test_image_representation()

Load On # 0 Resource Dimension Is  0.6933333333333334
Load On # 1 Resource Dimension Is  1.7933333333333334


In [8]:
canvas[1].shape

(20, 15)

In [1]:
[None] * 5

[None, None, None, None, None]

In [3]:
import numpy as np
canvas = np.zeros((2,10,10))

In [4]:
canvas[1,2,[0,2,3]]=0.3


In [5]:
canvas

array([[[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]],

       [[0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0.3, 0. , 0.3, 0.3, 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ,

In [ ]:
import cpuPower
intel = cpuPower.INTELCPU()


In [33]:
intel.get_power()
intel.get_power()


24.70908949168137

In [6]:
import numpy as np
back = np.arange(0,10,1)

In [3]:
back[:-1]=back[1:]

In [4]:
back

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 9])

In [7]:
import numpy as np
max_nw_vec=[10, 20]
np.divide(np.array(max_nw_vec), 2)


array([ 5., 10.])

In [4]:
len([None] * 10)
 

10

In [21]:
from scipy.stats import poisson
from collections import Counter
import matplotlib.pyplot as plt  # 
poi = poisson.rvs(5,size=1000)
rctd = Counter(poi)
plt.hist(poi)


[4,
 8,
 7,
 5,
 3,
 9,
 4,
 9,
 5,
 2,
 1,
 8,
 5,
 4,
 7,
 7,
 4,
 5,
 7,
 8,
 3,
 6,
 6,
 12,
 4,
 2,
 9,
 5,
 3,
 6,
 3,
 7,
 1,
 3,
 7,
 10,
 4,
 4,
 3,
 4,
 6,
 5,
 2,
 2,
 3,
 5,
 8,
 5,
 6,
 6,
 6,
 6,
 3,
 4,
 9,
 8,
 5,
 5,
 3,
 5,
 3,
 8,
 5,
 8,
 7,
 5,
 4,
 4,
 2,
 6,
 7,
 6,
 6,
 5,
 2,
 2,
 5,
 3,
 4,
 6,
 5,
 6,
 4,
 2,
 5,
 5,
 5,
 8,
 6,
 3,
 4,
 2,
 3,
 5,
 5,
 4,
 6,
 3,
 5,
 2]

In [14]:
for id in range(10,10):
    print(id)

In [13]:
import numpy as np
import math

avail = np.asarray([[5,7],[3,4],[0,0]])
av = avail/[5,7]
av


array([[1.        , 1.        ],
       [0.6       , 0.57142857],
       [0.        , 0.        ]])

In [20]:
av8 = av*8
av8 = av8.round()
def to_char(n):
    
avc = []

TypeError: sequence item 0: expected str instance, numpy.float64 found

In [2]:
char = bytes('\u2580',encoding="utf8")
lst = [c for c in char]
llst = []
for i in range(9):
    l=[]
    lst[2] +=1
    l=lst.copy()
    llst.append(l)
    cc = ''
for i in range(9):
    cc += str(bytes(llst[i]),encoding='utf8')

print(cc)
print(cc)


▁▂▃▄▅▆▇█▉
▁▂▃▄▅▆▇█▉


In [5]:
import numpy as np

In [11]:
bar = " ▁▂▃▄▅▆▇█"
bars = np.array([char for char in bar])

rr = np.array([[8, 3], [3, 5], [4, 7]])
cc = []
cc.append([bars[i] for i in rr[:,0]])
cc

[['█', '▃', '▄']]

In [6]:
arr = [[1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]]
lst = [' '.join([str(c) for c in i]) for i in arr]

lst

['1 2 3 4 5 6 7', '1 2 3 4 5 6 7']

In [5]:
' '.join([1, 2, 3, 4, 5, 6, 7])


TypeError: sequence item 0: expected str instance, int found

In [5]:
import torch

torch.torch_version.__version__

'2.1.0.dev20230308+cpu'

In [3]:
import torch
import torchvision.models as models

model = models.resnet18()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
compiled_model = torch.compile(model)

x = torch.randn(16, 3, 224, 224)
optimizer.zero_grad()
out = compiled_model(x)
out.sum().backward()
optimizer.step()
print(out.shape)

RuntimeError: Windows not yet supported for torch.compile

In [15]:
# 导入random模块
import random

# 定义一个函数，接受三个参数：min, max, t
def pick_numbers_r(min, max, t):
  # 创建一个空列表，用于存储选出的数
  result = []
  # 判断t是否大于max-min+1，如果是，抛出异常
  if t > max - min + 1:
    raise ValueError("t cannot be larger than max - min + 1")
  # 循环t次
  for i in range(t):
    # 随机生成一个数，范围在min和max之间（包含两端）
    num = random.randint(min, max)
    # 判断这个数是否已经在结果列表中，如果是，重新生成
    while num in result:
      num = random.randint(min, max)
    # 将这个数添加到结果列表中
    result.append(num)
  # 返回结果列表
  return result

# 调用函数，打印结果
print(pick_numbers_r(1, 10, 5))
%timeit pick_numbers_r(1, 10, 5)


[6, 2, 5, 8, 4]
4.18 µs ± 46.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
# 导入numpy模块
import timeit
import numpy as np

# 定义一个函数，接受三个参数：min, max, t


def pick_numbers_s(min, max, t):
  # 判断t是否大于max-min+1，如果是，抛出异常
  if t > max - min + 1:
    raise ValueError("t cannot be larger than max - min + 1")
  # 生成一个从min到max的整数数组
  array = np.arange(min, max + 1)
  # 随机打乱数组的顺序
  np.random.shuffle(array)
  # 返回数组的前t个元素
  return array[:t]


# 调用函数，打印结果
print(pick_numbers_s(1, 10, 5))
%timeit pick_numbers_s(1, 10, 5)


[3 2 8 7 1]
1.81 µs ± 13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
# 导入numpy模块
import numpy as np

# 定义一个函数，接受三个参数：min, max, t


def pick_numbers_c(min, max, t):
  # 判断t是否大于max-min+1，如果是，抛出异常
  if t > max - min + 1:
    raise ValueError("t cannot be larger than max - min + 1")
  # 用numpy的choice函数从min到max的整数中随机选取t个不重复的数
  result = np.random.choice(np.arange(min, max + 1), size=t, replace=False)
  # 返回结果数组
  return result


# 调用函数，打印结果
print(pick_numbers_c(1, 10, 5))

%timeit pick_numbers_c(1, 10, 5)


[8 4 6 9 7]
12.6 µs ± 60.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [28]:
min = np.random.randint(0, 15)
t = np.random.randint(3, 5)
array = np.arange(min, min+5)
np.random.shuffle(array)
array[:t]


array([3, 5, 1, 4])

In [1]:
import numpy as np
np.dot([2,3],[4,6])


26

In [2]:
8+18

26

In [3]:
np.random.normal()

-0.11528185322625457

In [4]:
[1,2,3,4,5,6][2,4,5]

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\Yuming\AppData\Local\Temp\ipykernel_13204\2954475527.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1,2,3,4,5,6][2,4,5]
C:\Users\Yuming\AppData\Local\Temp\ipykernel_13204\2954475527.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1,2,3,4,5,6][2,4,5]
C:\Users\Yuming\AppData\Local\Temp\ipykernel_13204\2954475527.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [1,2,3,4,5,6][2,4,5]


TypeError: list indices must be integers or slices, not tuple

In [5]:
import numpy as np
# 假设列表是 list1，子集是 list2
# 用 numpy 的 in1d 函数直接从 list1 中筛选出在 list2 中的元素，得到结果数组
list1 = [1,2,4,5,6,7]
list2 = [1,3,4,5]
result = np.array(list1)[np.in1d(list1, list2)]
# 打印结果数组
print(result)


[1 4 5]


In [7]:
np.in1d(list1, list2)


array([ True, False,  True,  True, False, False])

In [6]:
np.array(list1)[1,4,5]

IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

In [8]:
(5,6)

(5, 6)

In [1]:
import Job
jc = Job.JobCollection()
ijc = iter(jc)


In [2]:
next(ijc)

[[<Job.Job at 0x19324d04a30>,
 [<Job.Job at 0x19324d18d60>, <Job.Job at 0x19324d18dc0>],